In [19]:
import pandas as pd
from datetime import datetime
import numpy as np

In [20]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [21]:
def get_date(date):
    papi = [datetime(int(i[:4]),int(i[5:7]),int(i[8:10]),int(i[11:13]),int(i[14:16]),int(i[17:19])) for i in date]
    papi = [(i - min(papi)).total_seconds() for i in papi]
    papi = [i/max(papi) for i in papi]
    return papi

In [22]:
df['instock_date'] = get_date(df['instock_date'].values)
df['stock_update_date'] = get_date(df['stock_update_date'].values)
df['uk_date1'] = get_date(df['uk_date1'].values)
df['uk_date2'] = get_date(df['uk_date2'].values)
df_test['instock_date'] = get_date(df_test['instock_date'].values)
df_test['stock_update_date'] = get_date(df_test['stock_update_date'].values)
df_test['uk_date1'] = get_date(df_test['uk_date1'].values)
df_test['uk_date2'] = get_date(df_test['uk_date2'].values)

In [23]:
df['volumes'].fillna(1, inplace=True)
df_test['volumes'].fillna(1, inplace=True)

In [24]:
x_train = df.iloc[:-1000,1:-1].values
y_train = df.iloc[:-1000,-1].values
x_train_test = df.iloc[-1000:,1:-1].values
y_train_test = df.iloc[-1000:,-1].values
x_test = df_test.iloc[:,1:].values
ids = df_test.iloc[:,0]

x_train_w = []
y_train_w = []
x_train_test_w = []
y_train_test_w = []

x_train_o = []
y_train_o = []
x_train_test_o = []
y_train_test_o = []

for i in range(len(x_train)):
    if(x_train[i][-1] == 1):
        x_train_o.append(x_train[i][:-1])
        y_train_o.append(y_train[i])
    else:
        x_train_w.append(x_train[i])
        y_train_w.append(y_train[i])

for i in range(len(x_train_test)):
    if(x_train_test[i][-1] == 1):
        x_train_test_o.append(x_train_test[i][:-1])
        y_train_test_o.append(y_train_test[i])
    else:
        x_train_test_w.append(x_train_test[i])
        y_train_test_w.append(y_train_test[i])

x_train_w = np.array(x_train_w)
x_train_o = np.array(x_train_o)
y_train_w = np.array(y_train_w)
y_train_o = np.array(y_train_o)
x_train_test_w = np.array(x_train_test_w)
x_train_test_o = np.array(x_train_test_o)
y_train_test_w = np.array(y_train_test_w)
y_train_test_o = np.array(y_train_test_o)

from sklearn.preprocessing import StandardScaler
sc_w = StandardScaler()
x_train_w = sc_w.fit_transform(x_train_w)
x_train_test_w = sc_w.transform(x_train_test_w)

sc_o = StandardScaler()
x_train_o = sc_o.fit_transform(x_train_o)
x_train_test_o = sc_o.transform(x_train_test_o)

# print(len(x_train_test_o))
# print(len(x_train_test_w))

In [25]:
import keras
from keras.layers import Dense
from keras.models import Sequential

model_w = Sequential()
model_w.add(Dense(output_dim=100,init='uniform',activation='relu',input_dim=15))
model_w.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_w.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_w.add(Dense(output_dim=1,init='uniform',activation='linear'))

model_w.compile(optimizer="adam", loss="mean_squared_error",metrics=['accuracy'])
model_w.fit(x_train_w, y_train_w, epochs=100)
model_w.evaluate(x_train_test_w,y_train_test_w)

model_o = Sequential()
model_o.add(Dense(output_dim=100,init='uniform',activation='relu',input_dim=14))
model_o.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_o.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_o.add(Dense(output_dim=1,init='uniform',activation='linear'))

model_o.compile(optimizer="adam", loss="mean_squared_error",metrics=['accuracy'])
model_o.fit(x_train_o, y_train_o, epochs=100)
model_o.evaluate(x_train_test_o,y_train_test_o)

.3704e-04
Epoch 26/100
7297/7297 [==============================] - 1s 91us/step - loss: 978.8297 - accuracy: 1.3704e-04
Epoch 27/100
7297/7297 [==============================] - 1s 92us/step - loss: 962.6457 - accuracy: 1.3704e-04
Epoch 28/100
7297/7297 [==============================] - 1s 94us/step - loss: 947.9991 - accuracy: 2.7409e-04
Epoch 29/100
7297/7297 [==============================] - 1s 97us/step - loss: 935.6261 - accuracy: 0.0000e+00
Epoch 30/100
7297/7297 [==============================] - 0s 66us/step - loss: 904.5693 - accuracy: 1.3704e-04
Epoch 31/100
7297/7297 [==============================] - 0s 67us/step - loss: 878.4115 - accuracy: 2.7409e-04
Epoch 32/100
7297/7297 [==============================] - 1s 72us/step - loss: 1085.6784 - accuracy: 4.1113e-04
Epoch 33/100
7297/7297 [==============================] - 1s 100us/step - loss: 863.1693 - accuracy: 1.3704e-04
Epoch 34/100
7297/7297 [==============================] - 1s 100us/step - loss: 844.2971 - accuracy:

[5477.76802003457, 0.0]

In [26]:
y_pred = []
for x in x_test:
    if x[-1] == 1:
        p = x[:-1]
        p = np.array(p).reshape(1,-1)
        p = sc_o.transform(p)
        y_pred.append(model_o.predict(p))
    else:
        p = x
        p = np.array(p).reshape(1,-1)
        p = sc_w.transform(p)
        y_pred.append(model_w.predict(p))
for i in range(len(y_pred)):
    y_pred[i] = y_pred[i][0]

print(y_pred[10])

[32.456654]


In [28]:
import csv

with open('predictions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["gift_id", "price"])
    for i in range(len(y_pred)):
            writer.writerow([ids[i], y_pred[i][0]])